In [2]:
import sys
sys.path.insert(1, '/scratch/cinthiasouza/mv-text-summarizer')

import glob, os
import pandas as pd
import json
import spacy
import nltk
import numpy as np
import json
#import smogn
import seaborn as sns
import pickle

from bs4 import BeautifulSoup
from pysbd.utils import PySBDFactory
import math

from sumeval.metrics.rouge import RougeCalculator
rouge = RougeCalculator(stopwords=True, lang="en")
import matplotlib.pyplot as plt
import random

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from timeit import default_timer as timer 
from imblearn.over_sampling import SMOTE
from sklearn.metrics import matthews_corrcoef
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
#from tensorflow.keras.models import model_from_json

#nltk.download('averaged_perceptron_tagger')
#nltk.download('maxent_ne_chunker')
#nltk.download('words')
#nltk.download('punkt')
#nltk.download('stopwords')

#!python -m spacy download en_core_web_sm
#nlp_sm = spacy.load('en_core_web_sm')

import en_core_web_sm
nlp_md = en_core_web_sm.load()

import en_core_web_md
nlp_md = en_core_web_md.load()
#!python -m spacy download en_core_web_md
#nlp_md = spacy.load('en_core_web_md')

import warnings
warnings.filterwarnings("ignore")

path_base = "/scratch/cinthiasouza/mv-text-summarizer"


In [3]:
%load_ext autoreload
%autoreload 2

from src import preprocess
from src import extract_features
from src import tokenizer
from src import create_features_df
from src import transform_data
from src import loader
from src import utils
from src import ensemble_tree_models
from src import tunning_hyperparametrs as th
#from src import mlp_regressor
#from src import mlp_classifier
from src import summarization
from src import normalization
from src import ensemble_tree_models as classifiers
from src import utils_classification as utils_clf
from src import evaluate_classifiers as ev
from src import prepare_data
from src import display_results as dr

from joblib import Parallel, delayed
#from tensorflow.keras.utils import to_categorical
from src import pipeline_extract_features as pef
from joblib import Parallel, delayed

In [4]:
def main(name_section, initial_batch, embed_dim, batch_len, path_base, verbose):
    
    features_columns = ['sentences', 'text_rank', 'lex_rank', 'count_one_gram', 'count_two_gram',
       'count_three_gram', 'count_article_keywords', 'tf-isf',
       'position_score', 'paragraph_score', 'number_citations', 'length_score',
       'pos_score', 'ner_score', 'dist_centroid', 'articles']
    
    scores_columns = ['rouge_1', 'rouge_2', 'rouge_l', 'label', 'articles']
    
    embeddings_columns = [i for i in range(embed_dim)]
    embeddings_columns.append("article")
    

    """ if os.path.isfile('batches_{}.json'.format(batch_len)):

        print("Load Batch Files")
        with open('batches_{}.json'.format(batch_len)) as f:
            batches = json.load(f)

        batch_files = [value for key, value in batches.items()]
    else: 

        print("Creating Batch Files")
        batch_files = utils.create_batches(path_base, tam=batch_len)
        utils.save_batches(batch_files)
        

    batche_files = batch_files[initial_batch:]"""

    batche_files = os.listdir(path_base)

    print("Name section: " + name_section)
    vfunc = np.vectorize(pef.extract_features_file)
    
    print("Iniciando a extração de features...")
    
    for batch in batche_files:

    
        pef.extract_features_batches(
            vfunc, [batch], path_base, name_section=name_section, features_columns=features_columns,
            scores_columns=scores_columns, embeddings_columns=embeddings_columns, verbose=verbose)
 

In [5]:
initial_batch=0
#batche_files = batch_files[initial_batch:]

path = ["../../PMC002xxxxxx_pp", "../../PMC003xxxxxx_pp", "../../PMC005xxxxxx_pp", "../../PMC006xxxxxx_pp"]

sections = ['introduction', 'materials', 'conclusion']


In [6]:
from multiprocessing import Process, Queue

l1 = Queue()
p1 = Process(
    target=main, args=(
        sections[0], initial_batch, 300,
        700, path[0], False))

l2 = Queue()
p2 = Process(target=main,  args=(
        sections[1], initial_batch, 300,
        700, path[0], False))

l3 = Queue()
p3 = Process(target=main, args=(
        sections[2], initial_batch, 300,
        700, path[0], False))

l4 = Queue()
p4 = Process(
    target=main, args=(
        sections[0], initial_batch, 300,
        700, path[1], False))

l5 = Queue()
p5 = Process(target=main,  args=(
        sections[1], initial_batch, 300,
        700, path[1], False))

l6 = Queue()
p6 = Process(target=main, args=(
        sections[2], initial_batch, 300,
        700, path[1], False))

l7 = Queue()
p7 = Process(
    target=main, args=(
        sections[0], initial_batch, 300,
        700, path[2], False))

l8 = Queue()
p8 = Process(target=main,  args=(
        sections[1], initial_batch, 300,
        700, path[2], False))

l9 = Queue()
p9 = Process(target=main, args=(
        sections[2], initial_batch, 300,
        700, path[2], False))

l10 = Queue()
p10 = Process(
    target=main, args=(
        sections[0], initial_batch, 300,
        700, path[3], False))

l11 = Queue()
p11 = Process(target=main,  args=(
        sections[1], initial_batch, 300,
        700, path[3], False))

l12 = Queue()
p12 = Process(target=main, args=(
        sections[2], initial_batch, 300,
        700, path[3], False))
p1.start()   
p2.start()   
p3.start()
p4.start()   
p5.start()   
p6.start()
p7.start()   
p8.start()   
p9.start()
p10.start()   
p11.start()   
p12.start()

Name section: introduction
Iniciando a extração de features...Name section: materialsName section: conclusion


Iniciando a extração de features...Iniciando a extração de features...

Name section: materials
Iniciando a extração de features...Name section: introductionName section: conclusion


Iniciando a extração de features...Iniciando a extração de features...

Name section: introduction
Iniciando a extração de features...
Name section: materialsName section: conclusion

Iniciando a extração de features...
Iniciando a extração de features...
Name section: materialsName section: introductionName section: conclusion
Iniciando a extração de features...

Iniciando a extração de features...

Iniciando a extração de features...


In [4]:
from catboost import CatBoostClassifier
# Initialize data
cat_features = [0, 1]

train_data = [["a", "b", 1, 4, 5, 6],
              ["a", "b", 4, 5, 6, 7],
              ["c", "d", 30, 40, 50, 60]]
train_labels = [1, 1, 0]
eval_data = [["a", "b", 2, 4, 6, 8],
             ["a", "d", 1, 4, 50, 60]]

# Initialize CatBoostClassifier
model = CatBoostClassifier(iterations=2,
                           learning_rate=1,
                           depth=2, class_weights={0:0.95, 1:0.05})
# Fit model
model.fit(train_data, train_labels, cat_features)
# Get predicted classes
preds_class = model.predict(eval_data)
# Get predicted probabilities for each class
preds_proba = model.predict_proba(eval_data)
# Get predicted RawFormulaVal
preds_raw = model.predict(eval_data, prediction_type='RawFormulaVal')

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

0:	learn: 0.5403780	total: 245us	remaining: 245us
1:	learn: 0.4369467	total: 761us	remaining: 0us


In [5]:
import time

import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV


clf = xgb.XGBClassifier()

param_grid = {
        'silent': [False],
        'max_depth': [6, 10, 15, 20],
        'learning_rate': [0.001, 0.01, 0.1, 0.2, 0,3],
        'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'colsample_bytree': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'colsample_bylevel': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'min_child_weight': [0.5, 1.0, 3.0, 5.0, 7.0, 10.0],
        'gamma': [0, 0.25, 0.5, 1.0],
        'reg_lambda': [0.1, 1.0, 5.0, 10.0, 50.0, 100.0],
        'n_estimators': [100]}

fit_params = {'eval_metric': 'mlogloss',
              'early_stopping_rounds': 10,
              'eval_set': [(x_valid, y_valid)]}

rs_clf = RandomizedSearchCV(clf, param_grid, n_iter=20,
                            n_jobs=1, verbose=2, cv=2,
                            fit_params=fit_params,
                            scoring='neg_log_loss', refit=False, random_state=42)
print("Randomized search..")
search_time_start = time.time()
rs_clf.fit(x_train, y_train)
print("Randomized search time:", time.time() - search_time_start)

best_score = rs_clf.best_score_
best_params = rs_clf.best_params_
print("Best score: {}".format(best_score))
print("Best params: ")
for param_name in sorted(best_params.keys()):
    print('%s: %r' % (param_name, best_params[param_name]))

XGBoostError: [16:18:04] /tmp/build/80754af9/xgboost-split_1619724447847/work/dmlc-core/src/io/local_filesys.cc:104: LocalFileSystem.ListDirectory demo/data error: No such file or directory
Stack trace:
  [bt] (0) /scratch/cinthiasouza/anaconda3/lib/libxgboost.so(dmlc::io::LocalFileSystem::ListDirectory(dmlc::io::URI const&, std::vector<dmlc::io::FileInfo, std::allocator<dmlc::io::FileInfo> >*)+0xb14) [0x7f54efde1ae4]
  [bt] (1) /scratch/cinthiasouza/anaconda3/lib/libxgboost.so(dmlc::io::InputSplitBase::ConvertToURIs(std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > const&)+0x783) [0x7f54efddd373]
  [bt] (2) /scratch/cinthiasouza/anaconda3/lib/libxgboost.so(dmlc::io::InputSplitBase::InitInputFileInfo(std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > const&, bool)+0x46) [0x7f54efdde156]
  [bt] (3) /scratch/cinthiasouza/anaconda3/lib/libxgboost.so(dmlc::io::InputSplitBase::Init(dmlc::io::FileSystem*, char const*, unsigned long, 